# **Анализ оттока клиентов банка**

# Введение

**Целью** данного исследования является выявление сегментов клиентов банка, которые в наибольшей степени склонны уходить, выявление ключевых причин оттока, предложение мер для удержания клиентов, построение и сравнительный анализ моделей машинного обучения по прогнозированию ухода клиента.

**Задачи:**
1) Загрузка датасета и импорт библиотек
2) Разведочный анализ данных (EDA)
3) Составление портретов клиентов
4) Проверка статистических гипотез
5) Составление сегментов киентов в соответствии с их подверженности уходу из банка
6) Подготовка рекомендаций для бизнеса
7) Построение модели машинного обучения
   1. Подготовка датасета
  

# 1. Загрузка датасета и импорт библиотек

Для данного исследования использовался датасет с Kaggle: [Bank Customer Churn Prediction dataset](http://www.kaggle.com/datasets/shantanudhakadd/bank-customer-churn-prediction).

Датасет представляет собой набор данных о пользователях банка США.

* **Customer ID**: ID для каждого клиента
* **Surname**: фамилия клиента
* **Credit Score**: кредитный рейтинг 
* **Geography**: страна проживания 
* **Gender**: пол 
* **Age**: возраст
* **Tenure**: количество лет, в течение которых пользователь является клиентом банка
* **Balance**: баланс счета
* **NumOfProducts**: количество банковских продуктов, которыми пользуется клиент (сберегательный счет, кредитная карта и др.)
* **HasCrCard**: имеет ли клиент кредитную карту
* **IsActiveMember**: активный ли клиент
* **EstimatedSalary**: предполагаемая зарплата клиента 
* **Exited**: ушел ли клиент (целевая переменная)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import shap

import warnings
warnings.filterwarnings("ignore")

In [2]:
# сохраним файл с данными в переменную df
df = pd.read_csv('/kaggle/input/bank-customer-churn-prediction/Churn_Modelling.csv')

In [3]:
# выведем на экран первые 5 строк датасета 
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.shape

(10000, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [6]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Статистические характеристики каждого столбца датасета соответствуют целям анализа - некорректных значений нет.

# 2. Разведочный анализ данных

In [7]:
# работа с пропусками
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

Как видим, в датасете нет пропусков

In [8]:
# проверим датасет на наличие дубликатов
print(df.duplicated().sum())
print(df['CustomerId'].duplicated().sum())

0
0


In [9]:
# проверим на корректность содержимое столбцов с категориальными переменными
print(df['Geography'].unique())
print(df['Gender'].unique())
print(df['HasCrCard'].unique())
print(df['IsActiveMember'].unique())
print(df['Exited'].unique())

['France' 'Spain' 'Germany']
['Female' 'Male']
[1 0]
[1 0]
[1 0]


In [10]:
# проверка на выбросы
pd.set_option("display.float_format", "{:.2f}".format)
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00
mean,5000.50,15690940.57,650.53,38.92,5.01,76485.89,1.53,0.71,0.52,100090.24,0.20
std,2886.90,71936.19,96.65,10.49,2.89,62397.41,0.58,0.46,0.50,57510.49,0.40
min,1.00,15565701.00,350.00,18.00,0.00,0.00,1.00,0.00,0.00,11.58,0.00
25%,2500.75,15628528.25,584.00,32.00,3.00,0.00,1.00,0.00,0.00,51002.11,0.00
50%,5000.50,15690738.00,652.00,37.00,5.00,97198.54,1.00,1.00,1.00,100193.91,0.00
75%,7500.25,15753233.75,718.00,44.00,7.00,127644.24,2.00,1.00,1.00,149388.25,0.00
max,10000.00,15815690.00,850.00,92.00,10.00,250898.09,4.00,1.00,1.00,199992.48,1.00


Заметим, что в столбце EstimatedSalary минимальное значение 11.58, что является аномально низкой заработной платой (минимульная зарплата в США - 1160 долларов в месяц). Проверим, является ли это выбросом.

In [11]:
df[df['EstimatedSalary']<1160]['CustomerId'].count()

67

Менее 1% клиентов имеют аномально низкую заработную плату, поэтому удалим эти строки.


In [12]:
df = df[df['EstimatedSalary']>1160]